# Computational Geometry in Python
reference: 'Discrete and Computational Geometry' (2011) Devadoss and O'Rourke.

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
import matplotlib
%matplotlib inline
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1) Introduction: What is Computational Geometry? Why is it important?

## 2) Polygons